# MOBIL模型仿真

In [14]:
import math
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import random
dt=0.1#步长
Cn=0#换道次数
C=[]
A_i=4
v_f=30
δ=2
l=6
s_0=8
s_1=0
T_i=1.1
b_i=4
Δa=0.39
a_bias=0
b_safe=-4

In [15]:
class Vehicle:#定义
    def __init__(self,ID=0,LID=1,T=10,S=float("inf"),V=v_f,A=0,X=0,flag=0,stop=0,p=0.1):
        self.ID=ID
        self.LID=[LID for t in range(T)]
        self.FID=-1
        self.BID=-1
        self.FCID=-1
        self.BCID=-1
        self.FFlag=0
        self.CFlag=0
        self.s=[S for t in range(T)]
        self.v=[V for t in range(T)]
        self.a=[A for t in range(T)]
        self.x=[X for t in range(T)]
        self.flag=flag#是否在路段内
        self.stop=stop#是否停止
        self.p=p#利他因子

In [16]:
class Lane:
    def __init__(self,ID=1,LL=1000,T=1*60*60,Iq=1000/3600,flag=1,dm=0):
        self.flag=flag#末端能否通行
        self.ID=ID
        self.LL=LL
        self.Iq=Iq
        if self.flag!=0:
            self.VID=0
            self.Vehicles=[Vehicle(n,ID,T) for n in range(int(T*Iq))]
        if self.flag==0:
            self.VID=1#起始车辆ID设置为1
            #初始位置和利他因子设置
            self.Vehicles=[Vehicle(ID=n,LID=ID,T=T,X=dm,p=0) for n in range(int(T*Iq))]
            #设置一辆常停车在路端
            self.Vehicles[0].stop=1
            self.Vehicles[0].flag=1
            self.Vehicles[0].x=[LL+dm for t in range(T)]

In [17]:
def IDM(s=s_0,v1=0,v0=0):#IDM模型求加速度
    if s-l<=0:
        s=6.01
    a=A_i*(1-math.pow((v1/v_f),δ)-math.pow(((Sit(v1,v0)-l)/(s-l)),2))
    return a

In [18]:
def Sit(v1=0,v0=0):#期望车距函数
    Sx=s_0+s_1*math.pow((v1/v_f),0.5)+T_i*v1+(v1*(v1-v0))/(2*math.pow(A_i*b_i,0.5))
    return Sx

In [19]:
#换道需求+安全换道
def DS(a0=0,a1=0,a2=0,ac2=0,s1c=s_0,s2c=s_0,sc2c=s_0,v0=0,vc0=0,v1=0,v2=0,vc2=0,p=1):
    a1c=IDM(s1c,v1,vc0)
    a2c=IDM(s2c,v2,v0)
    ac2c=IDM(sc2c,vc2,v1)
    D=a1c-a1+p*(a2c-a2+ac2c-ac2)
    if D>Δa+a_bias and ac2c>b_safe:
        return 1
    else:
        return 0

In [20]:
def Poisson(lam):#泊松近似
    u = random.uniform(0,1)
    p = 2.718281828459045**(-lam)
    f = p
    if u < f:
        return 0
    else:
        return 1

In [21]:
def Simulation(ln=2,ll=1000,T=1*60*60,Iq=1000/3600):#仿真
    global Cn
    global C
    #初始化
    T=int(T/dt)
    #主路
    Lanes=[Lane(n+1,ll,T,Iq) for n in range(ln)]
    #匝道
    MLane=Lane(0,300,T,500/3600,0,7500)
    Lanes.insert(0,MLane)
    for i in range(3):
        print(Lanes[i].ID)
    Vehicles=[MLane.Vehicles[0]]#路段内的车
    #print(Vehicles[0].x[0],Vehicles[0].v[0],Vehicles[0].LID[0])
    for t in range(T-1):
        #到达
        for L in Lanes:
            if Poisson(L.Iq*dt):
                if L.VID<T*L.Iq*dt:
                    V=L.Vehicles[L.VID]
                    V.flag=1
                    Vehicles.append(V)
                    L.VID+=1
                    
        #跟驰判断与匹配
        for V in Vehicles:
            if V.stop==0:
                if V.LID[t]==0 and V.ID!=0 and 1070>V.x[t]>=1030 and t%10==0:
                    print(V.ID,V.x[t],V.v[t],V.a[t],V.FID,Vehicles[0].LID[t])
                V.FFlag=0
                for F in Vehicles:
                    if F.LID[t]==V.LID[t] and F.x[t]>V.x[t]:
                        V.FFlag=1
                        if F.x[t]-V.x[t]<V.s[t]:
                            V.FID=Vehicles.index(F)
                            V.s[t]=F.x[t]-V.x[t]
                if V.FFlag==0:
                    V.FID=-1
        #跟驰
        for V in Vehicles:
            if V.stop==0:
                if V.FFlag==1:
                    V.x[t+1]=V.x[t]+V.v[t]*dt+0.5*V.a[t]*dt*dt
                    V.v[t+1]=max(V.v[t]+V.a[t]*dt,0)
                    V.a[t+1]=IDM(V.s[t],V.v[t],Vehicles[V.FID].v[t])
                else:
                    V.x[t+1]=V.x[t]+V.v[t]*dt+0.5*V.a[t]*dt*dt
                    V.v[t+1]=max(V.v[t]+V.a[t]*dt,0)
                    V.a[t+1]=IDM(float("inf"),V.v[t],v_f)
        
        #后车查询
        for V in Vehicles:
            for B in Vehicles:
                if B.FID==Vehicles.index(V):
                    V.BID=Vehicles.index(B)
                    break
            V.BID=-1
        
        #相邻车道前车判断与匹配
        for V in Vehicles:
            if V.stop==0:
                ts=float("inf")
                V.FCID=-1
                for F in Vehicles:
                    if ((F.LID[t]==1 and V.LID[t]==0) or (F.LID[t]==2 and V.LID[t]==1) or (F.LID[t]==1 and V.LID[t]==2)) and 0<F.x[t]-V.x[t]<ts:
                        V.FCID=Vehicles.index(F)
                        ts=F.x[t]-V.x[t]
        
        #相邻车道后车判断与匹配
        for V in Vehicles:
            if V.stop==0:
                ts=float("inf")
                V.BCID=-1
                for B in Vehicles:
                    if ((B.LID[t]==1 and V.LID[t]==0) or (B.LID[t]==2 and V.LID[t]==1) or (B.LID[t]==1 and V.LID[t]==2)) and 0<V.x[t]-B.x[t]<ts:
                        V.BCID=Vehicles.index(B)
                        ts=V.x[t]-B.x[t]
        
        #换道判断
        for V in Vehicles:
            if V.stop==0:
                a0=0;a1=V.a[t];a2=0;ac2=0;s1c=float("inf");s2c=float("inf");sc2c=float("inf");v0=0;vc0=0;v1=V.v[t];v2=0;vc2=0;p=V.p
                if V.FID>=0:
                    F=Vehicles[V.FID]
                    a0=F.a[t]
                    v0=F.v[t]
                if V.BID>=0:
                    B=Vehicles[V.BID]
                    a2=B.a[t]
                    v2=B.v[t]
                    if V.FID>=0:
                        s2c=Vehicles[V.FID].x[t]-B.x[t]
                if V.FCID>=0:
                    FC=Vehicles[V.FCID]
                    s1c=FC.x[t]-V.x[t]
                    vc0=FC.v[t]
                if V.BCID>=0:
                    BC=Vehicles[V.BCID]
                    ac2=BC.a[t]
                    vc2=BC.v[t]
                    sc2c=V.x[t]-BC.x[t]
                if DS(a0,a1,a2,ac2,s1c,s2c,sc2c,v0,vc0,v1,v2,vc2,p):
                    if V.LID[t]==1:
                        V.CFlag=2
                    else:
                        V.CFlag=1
                else:
                    V.CFlag=0
        #换道
        for V in Vehicles:
            if V.stop==0:
                while V.FID!=-1 and V.CFlag!=0:#检测100m内有无换道行为，有则推迟换道
                    if Vehicles[V.FID].x[t]-V.x[t]>100:
                        break
                    if Vehicles[V.FID].CFlag!=0:
                        V.CFlag=0
                        break
                    V.FID=Vehicles[V.FID].FID
                if V.CFlag!=0 and (V.FID<0 or Vehicles[V.FID].CFlag==0):
                    if t>300/dt:
                        Cn+=1
                        C.append((V.x[t],V.LID[t]))
                    if 50<t<100:
                        print(V.LID[t],V.CFlag)
                    for tt in range(t,T-1):
                        V.LID[tt]=V.CFlag
                        
                        
        
        #出界处理
        for V in Vehicles:
            if V.x[t]>ll or V.x[t]<0:
                V.flag=0
                del Vehicles[Vehicles.index(V)]
    
    print("ok")
    return Lanes

In [1]:
def Visual(Results,ll,ln,T):#可视化
    x=[100*int(i/3) for i in range(int(3*ll/100+3))]
    y=[i%3 for i in range(int(3*ll/100+3))]
    #print(y)
    s=[0 for i in range(int(3*ll/100+3))]
    for c in C:
        if 0<=c[0]<=ll:
            s[3*int(c[0]/100)+c[1]]+=1
        #if c[1]!=1:
            #print(c[0],c[1],3*int(c[0]/100)+c[1],(3*int(c[0]/100)+c[1])%3)
    plt.rcParams["font.family"] = 'Arial Unicode MS'
    plt.figure(figsize=(15,1))
    plt.xlim([0,10000])
    plt.scatter(x,y,s)
    fig=plt.figure(figsize=(15,1))
    plt.title("车道-位移")
    plt.xlim([0,10000])
    ims=[]
    for t in range(int(T/dt)-1):
        if 3600<t<6000 and t%10==0:
            x=[0,0,0]
            y=[0,0,0]
            for i in range(3):
                x[i]=[Results[i].Vehicles[n].x[t] for n in range(int(T*Results[i].Iq))]
                y[i]=[Results[i].Vehicles[n].LID[t] for n in range(int(T*Results[i].Iq))]
            X=x[0]+x[1]+x[2]
            Y=y[0]+y[1]+y[2]
            im=plt.scatter(X,Y,c='k').findobj()
            ims.append(im)
    animation = ani.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000)
    animation.save("test.gif",writer='pillow')

In [29]:
R=Results[1].Vehicles[42]
i=122
print(R.LID[i],R.x[i],R.v[i],R.a[i],R.FID,R.FCID)
print(R.LID)

1 0 30 0 91 93
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [24]:
print(Cn*dt)
print(Results[0].VID,Results[1].VID,Results[2].VID)

1238.3000000000002
92 200 198


In [23]:
ln=2
ll=10*1000
T=0.2*60*60
Iq=1000/3600
Results=Simulation(ln,ll,T,Iq)

0
1
2
1 2
2 1
1 2
2 1
1 2
2 1
1 2
2 1
1 2
1 2
2 1
1 2
2 1
1 2
2 1
1 2
2 1
1 2
2 1
1 2
1 2
2 1
ok


In [2]:
Visual(Results,ll,ln,T)

NameError: name 'Results' is not defined

In [9]:
R=Results[0]
C=R.Vehicles[0]
print(C.p,C.stop,C.flag)

0 1 1
